In [52]:
import pandas as pd
df = pd.read_csv("../Data/2019/201901_LDA.csv")
df = df.append(pd.read_csv("../Data/2019/201902_LDA.csv"), ignore_index = True)
df = df.append(pd.read_csv("../Data/2019/201903_LDA.csv"), ignore_index = True)
df = df.append(pd.read_csv("../Data/2019/201904_LDA.csv"), ignore_index = True)
df = df.append(pd.read_csv("../Data/2019/201905_LDA.csv"), ignore_index = True)
df = df.append(pd.read_csv("../Data/2019/201906_LDA.csv"), ignore_index = True)


In [53]:
df = df.drop('topics', axis = 1)
df = df.dropna()

In [3]:
N_list = df['N_list']
# N_list = N_list.head(100)

In [4]:
corpus = []
for Nouns in N_list:
    Nouns = Nouns.replace("[", "").replace("]", "").replace("'", "")
    corpus.append(" ".join(Nouns.split(', ')))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

mylist = []
for i, sent in enumerate(corpus):
#     print('====== document[%d] ======' % i)
    mylist.append([(token, sp_matrix[i, word2id[token]]) for token in sent.split()])
#     print( [ (token, sp_matrix[i, word2id[token]]) for token in sent.split() ] )
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [15]:
docs = []
for m in mylist:
    words = []
    values = []
    for noun in m:
        words.append(noun[0])
        values.append(noun[1])
    doc = pd.DataFrame({"word":words, "value":values})
    doc = doc.drop_duplicates(['word'])
    doc = doc.sort_values(by = 'value', ascending = False)
    docs.append(doc)

In [50]:
topics = []
for doc in docs:
    df2 = pd.DataFrame(doc)
    topic = []
    for i in range(len(df2)):
        topic.append(df2.iloc[i]['word'])
        if(i == 9):
            break
    topics.append(', '.join(topic))

In [54]:
df['topics'] = topics

In [57]:
df.to_csv("../Data/2019/2019_TFIDF.csv", encoding = 'utf-8-sig')